# Faster autoscaling on Amazon SageMaker realtime endpoints with inference components (Application Autoscaling)

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)


---

In this notebook we show how the new faster autoscaling feature helps scale sagemaker inference endpoints by almost 6x faster than earlier.

We deploy Meta's `Llama3-8B-Instruct` model to an Amazon SageMaker realtime endpoint using Text Generation Inference (TGI) Deep Learning Container (DLC) and apply <span style='color:green'><b>Application Autoscaling</b></span> scaling policies to the endpoint.


<div class="alert alert-block alert-warning">
    Please select <b>m5.2xlarge</b> or larger instance types when running this on Amazon SageMaker Notebook Instance.<br/>
    Select <b>conda_pytorch_p310</b> kernel when running this notebook on Amazon SageMaker Notebook Instance. <br/><br/>
    Ensure python version for the kernel is <b>3.10.x</b> (3.11 is not supported). <br/>
</div>

---

## Prerequisites



<div style="border: 1px solid #f00; border-radius: 5px; padding: 10px; background-color: #fee;">
Before using this notebook please ensure you have access to an active access token from HuggingFace and have accepted the license agreement from Meta.

- **Step 1:** Create user access token in HuggingFace (HF). Refer [here](https://huggingface.co/docs/hub/security-tokens) on how to create HF tokens.
- **Step 2:** Login to [HuggingFace](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/tree/main) and navigate to *Meta-Llama-3-8B-Instruct** home page.
- **Step 3:** Accept META LLAMA 3 COMMUNITY LICENSE AGREEMENT by following the instructions [here](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/tree/main)
- **Step 4:** Wait for the approval email from META (Approval may take any where b/w 1-3 hrs)
</div>

Install packages using uv, an extremely fast python package installer\
Read more about uv here <https://astral.sh/blog/uv>

In [1]:
# ensure python version of the selected kernel is not greater than 3.10
!python --version

Python 3.10.14


In [2]:
!pip install uv && uv pip install -U ipywidgets
!uv pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 110.8 MB/s eta 0:00:0000:010:01
Resolved 22 packages in 63ms                                         
Prepared 14 packages in 326ms                                            
Uninstalled 14 packages in 372ms
Installed 14 packages in 49ms                               
 - exceptiongroup==1.2.0 (from file:///home/conda/feedstock_root/build_artifacts/exceptiongroup_1704921103267/work)
 + exceptiongroup==1.2.2
 - ipython==8.22.2 (from file:///home/conda/feedstock_root/build_artifacts/ipython_1709559745751/work)
 + ipython==8.26.0
 - ipywidgets==8.1.2 (from file:///home/conda/feedstock_root/build_artifacts/ipywidgets_1707427226251/work)
 + ipywidgets==8.1.3
 - jedi==0.18.2 (from file:///home/conda/feedstock_root/build_artifacts/jedi_1669134318875/work)
 + jedi==0.19.1
 - jupyterlab-widgets==3.0.10 (from file:///home/conda/feedstock_root/build_artifacts/jupyterlab_widgets_1707421892171/work)
 + jupyterlab-widgets==3.0.11
 - matplotlib

In [3]:
# restart kernel
from IPython.core.display import HTML

HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
# load rich extension
%load_ext rich

In [5]:
import sys
import os

import time
from getpass import getpass
import boto3
import sagemaker
from rich import print
from sagemaker.deserializers import JSONDeserializer
from sagemaker.huggingface import get_huggingface_llm_image_uri
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Initiate sagemaker session

In [6]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = sess.sagemaker_client
sagemaker_runtime_client = sess.sagemaker_runtime_client
cloudwatch_client = boto3.client("cloudwatch", region_name=region)

hf_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# retrieve the llm image uri
# tgi_dlc = f"763104351884.dkr.ecr.{region}.amazonaws.com/huggingface-pytorch-tgi-inference:2.1-tgi2.0-gpu-py310-cu121-ubuntu22.04"
tgi_dlc = get_huggingface_llm_image_uri("huggingface", version="2.0.0")

print(f"TGI DLC: \n[b i green]{tgi_dlc}[/b i green]")
print(f"Region: [b blue]{region}[/b blue]")
print(f"Role: [b red]{role}[/b red]")

TGI DLC: 
763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.1-tgi2.0.0-gpu-py310-cu121-ubunt
u22.04

Region: us-east-1

Role: arn:aws:iam::057716757052:role/gen_ai_gsmoon

## Create Endpoint

1. Create `EndpointConfiguration`
2. Create Endpoint

In [7]:
# Set an unique endpoint config name
prefix = sagemaker.utils.unique_name_from_base("llama3")
print(f"prefix: {prefix}")

endpoint_config_name = f"{prefix}-endpoint-config"
print(f"Endpoint config name: {endpoint_config_name}")

# Set varient name and instance type for hosting
variant_name = "AllTraffic"
instance_type = "ml.g5.2xlarge"
model_data_download_timeout_in_seconds = 3600
container_startup_health_check_timeout_in_seconds = 3600

initial_instance_count = 1
max_instance_count = 2
print(f"Initial instance count: {initial_instance_count}")
print(f"Max instance count: {max_instance_count}")

epc_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ExecutionRoleArn=role,
    ProductionVariants=[
        {
            "VariantName": variant_name,
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": model_data_download_timeout_in_seconds,
            "ContainerStartupHealthCheckTimeoutInSeconds": container_startup_health_check_timeout_in_seconds,
            "ManagedInstanceScaling": {
                "Status": "ENABLED",
                "MinInstanceCount": initial_instance_count,
                "MaxInstanceCount": max_instance_count,
            },
            "RoutingConfig": {"RoutingStrategy": "LEAST_OUTSTANDING_REQUESTS"},
        }
    ],
)
print(epc_response)

prefix: llama3-1722146573-5d08

Endpoint config name: llama3-1722146573-5d08-endpoint-config

Initial instance count: 1

Max instance count: 2

{
    'EndpointConfigArn': 
'arn:aws:sagemaker:us-east-1:057716757052:endpoint-config/llama3-1722146573-5d08-endpoint-config',
    'ResponseMetadata': {
        'RequestId': '3869917a-91f4-47aa-9a27-9dff94633eb7',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amzn-requestid': '3869917a-91f4-47aa-9a27-9dff94633eb7',
            'content-type': 'application/x-amz-json-1.1',
            'content-length': '119',
            'date': 'Sun, 28 Jul 2024 06:02:53 GMT'
        },
        'RetryAttempts': 0
    }
}

In [8]:
# Set a unique endpoint name
endpoint_name = f"{prefix}-endpoint"

ep_response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)
# print(ep_response)
print(f"Creating endpoint: [b blue]{endpoint_name}...")
sess.wait_for_endpoint(endpoint_name)

Creating endpoint: llama3-1722146573-5d08-endpoint...

-----!


{
    'EndpointName': 'llama3-1722146573-5d08-endpoint',
    'EndpointArn': 'arn:aws:sagemaker:us-east-1:057716757052:endpoint/llama3-1722146573-5d08-endpoint',
    'EndpointConfigName': 'llama3-1722146573-5d08-endpoint-config',
    'ProductionVariants': [
        {
            'VariantName': 'AllTraffic',
            'CurrentInstanceCount': 1,
            'DesiredInstanceCount': 1,
            'ManagedInstanceScaling': {
                'Status': 'ENABLED',
                'MinInstanceCount': 1,
                'MaxInstanceCount': 2
            },
            'RoutingConfig': {'RoutingStrategy': 'LEAST_OUTSTANDING_REQUESTS'}
        }
    ],
    'EndpointStatus': 'InService',
    'CreationTime': datetime.datetime(2024, 7, 28, 6, 3, 19, 975000, tzinfo=tzlocal()),
    'LastModifiedTime': datetime.datetime(2024, 7, 28, 6, 6, 10, 61000, tzinfo=tzlocal()),
    'ResponseMetadata': {
        'RequestId': '0efc7190-5016-4331-a664-366b08e37262',
        'HTTPStatusCode': 200,
        'HTTPHea

## Deploy model

Create and deploy model using Amazon SageMaker HuggingFace TGI DLC

<https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy>

<div class="alert alert-block alert-warning">
<b>NOTE:</b> Remember to copy your Hugging Face Access Token from <a href="https://hf.co/">https://hf.co/</a> before running the below cell.<br/><br/>
Refer <a href="https://huggingface.co/docs/hub/security-tokens">here</a> to learn about creating HF tokens.
</div>

## Configure container and environment 

In [9]:
# print ecr image uri
print(f"llm image uri: [b green]{tgi_dlc}")

HF_TOKEN = os.getenv("HUGGING_FACE_HUB_TOKEN") or getpass("Enter HUGGINGFACE Access Token: ")


llama3model = {
    "Image": tgi_dlc,
    "Environment": {
        "HF_MODEL_ID": "meta-llama/Meta-Llama-3-8B-Instruct",  # model_id from hf.co/models
        "SM_NUM_GPUS": "1",  # Number of GPU used per replica
        "MAX_INPUT_LENGTH": "2048",  # Max length of input text
        "MAX_TOTAL_TOKENS": "4096",  # Max length of the generation (including input text)
        "MAX_BATCH_TOTAL_TOKENS": "8192",  # Limits the number of tokens that can be processed in parallel during the generation
        "MESSAGES_API_ENABLED": "true",  # Enable the messages API
        "HUGGING_FACE_HUB_TOKEN": HF_TOKEN,
    },
}

# create Model
deployment_name = "sm"
model_name = f"{deployment_name}-model-llama3"

print(f"Creating model: [b green]{model_name}...")
model_response = sagemaker_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    Containers=[llama3model],
)

print(model_response)

llm image uri: 
763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.1-tgi2.0.0-gpu-py310-cu121-ubunt
u22.04

Creating model: sm-model-llama3...

{
    'ModelArn': 'arn:aws:sagemaker:us-east-1:057716757052:model/sm-model-llama3',
    'ResponseMetadata': {
        'RequestId': '38bc404b-df57-4db3-9d10-0f37a404fa34',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amzn-requestid': '38bc404b-df57-4db3-9d10-0f37a404fa34',
            'content-type': 'application/x-amz-json-1.1',
            'content-length': '77',
            'date': 'Sun, 28 Jul 2024 06:10:30 GMT'
        },
        'RetryAttempts': 0
    }
}

In [10]:
# Deploy model to Amazon SageMaker Inference Component
inference_component_name_llama3b = f"{prefix}-IC-llama3b"
variant_name = "AllTraffic"

ic_response = sagemaker_client.create_inference_component(
    InferenceComponentName=inference_component_name_llama3b,
    EndpointName=endpoint_name,
    VariantName=variant_name,
    Specification={
        "ModelName": f"{deployment_name}-model-llama3",
        "ComputeResourceRequirements": {
            "NumberOfAcceleratorDevicesRequired": 1,
            "NumberOfCpuCoresRequired": 1,
            "MinMemoryRequiredInMb": 1024,
        },
    },
    RuntimeConfig={"CopyCount": 1},
)

# print(ic_response)

# Wait for IC to come InService
print(f"InferenceComponent [b magenta]{inference_component_name_llama3b}...")
while True:
    desc = sagemaker_client.describe_inference_component(
        InferenceComponentName=inference_component_name_llama3b
    )
    status = desc["InferenceComponentStatus"]
    print(status)
    sys.stdout.flush()
    if status in ["InService", "Failed"]:
        break
    time.sleep(30)

InferenceComponent llama3-1722146573-5d08-IC-llama3b...

Creating

Creating

Creating

Creating

...

Creating

Creating

Creating

Creating

InService

## Inference

Invoke and test endpoint using messages API. Refer to HF [Messages API](https://huggingface.co/docs/text-generation-inference/messages_api) for more info.

In [13]:
endpoint_name, inference_component_name_llama3b

('llama3-1722146573-5d08-endpoint', 'llama3-1722146573-5d08-IC-llama3b')

In [11]:
# create predictor object
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    component_name=inference_component_name_llama3b,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

In [12]:
# Prompt to generate
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is deep learning?"},
]

# Generation arguments
parameters = {
    "model": hf_model_id,  # model id is required
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 512,
    "stop": ["<|eot_id|>"],
}

chat = predictor.predict({"messages": messages, **parameters})

# Unpack and print response
print(chat["choices"][0]["message"]["content"].strip())

Deep learning is a subfield of machine learning that is inspired by the structure and function of the human brain. 
It involves the use of artificial neural networks, which are composed of multiple layers of interconnected nodes or
"neurons," to analyze and interpret data.

In traditional machine learning, algorithms are designed to learn from data by identifying patterns and 
relationships between inputs and outputs. However, these algorithms are typically limited to learning from shallow 
representations of data, such as features or attributes.

Deep learning, on the other hand, allows machines to learn from complex, hierarchical representations of data, such
as images, speech, and text. This is achieved through the use of multiple layers of neural networks, each of which 
is trained to recognize and extract more abstract features from the data.

The key characteristics of deep learning are:

1. Hierarchical representations: Deep learning models learn to represent data in a hierarchical manner, with each 
layer building on the previous one to create a more abstract and complex representation of the data.
2. Multiple layers: Deep learning models typically consist of multiple layers of neural networks, each of which is 
trained to perform a specific task, such as feature extraction or classification.
3. Large amounts of data: Deep learning models require large amounts of data to train, as they need to learn from 
the patterns and relationships present in the data.
4. Computational power: Deep learning models require significant computational power to train, as they involve 
complex mathematical calculations and iterative optimization processes.

Some of the key applications of deep learning include:

1. Computer vision: Deep learning has been used to develop systems that can recognize and interpret images and 
videos, such as facial recognition, object detection, and image classification.
2. Natural language processing: Deep learning has been used to develop systems that can understand and generate 
human language, such as speech recognition, language translation, and text summarization.
3. Speech recognition: Deep learning has been used to develop systems that can recognize and transcribe spoken 
language, such as voice assistants and speech-to-text systems.
4. Robotics: Deep learning has been used to develop systems that can control and interact with robots, such as 
autonomous vehicles and robotic arms.

Some of the key benefits of deep learning include:

1. Improved accuracy: Deep learning models can achieve high levels of accuracy on complex tasks, such as image 
recognition and speech recognition.
2. Flexibility: Deep learning models can be trained on a wide range of data types and can be adapted to new tasks 
and domains.
3. Scalability: Deep learning models can be trained on large datasets and can be

## Apply Autoscaling policies to the endpoint

Apply Application Autoscaling Policy to endpoint

1. Register Scalable Target

In [14]:
as_min_capacity = 1
as_max_capacity = 2

resource_id = f"inference-component/{inference_component_name_llama3b}"

autoscaling_client = boto3.client("application-autoscaling", region_name=region)

# Register scalable target
scalable_target = autoscaling_client.register_scalable_target(
    ServiceNamespace="sagemaker",
    ResourceId=resource_id,
    ScalableDimension="sagemaker:inference-component:DesiredCopyCount",
    MinCapacity=as_min_capacity,
    MaxCapacity=as_max_capacity,  # Replace with your desired maximum instances
)

scalable_target_arn = scalable_target["ScalableTargetARN"]
print(f"Resource ID: [b blue]{resource_id}")
print(f"Scalable_target_arn:\n[b green]{scalable_target_arn}")

Resource ID: inference-component/llama3-1722146573-5d08-IC-llama3b

Scalable_target_arn:
arn:aws:application-autoscaling:us-east-1:057716757052:scalable-target/056mffd3d71426ed4b47997cf648b07f9600

## Use the latest high-resolution Metrics to trigger auto-scaling

- New feature introduces a new <span style='color:green'><b>PredefinedMetricType</b></span> for scaling policy configuration i.e. <span style='color:green'><b>SageMakerVariantConcurrentRequestsPerModelHighResolution</b></span> to trigger scaling actions.
- Creating a scaling policy with this metric type will create cloudwatch alarms that track a new metric called <span style='color:green'><b>ConcurrentRequestsPerModel</b></span>.
- These high-resolution metrics are published at sub-minute intervals (10s intervals to CW + any additional jitter + delays)
- We should observe significant improvement in scale out times with this new metric


### Steps to create Application autoscaling policy

- Create scaling policy
  - Set `PolicyType` to `TargetTrackingScaling`
  - Set `TargetValue` to `5.0`. i.e., Scaling triggers when endpoint receives 5 `ConcurrentRequestsPerModel`
  - Set `PredefinedMetricType` to `SageMakerVariantConcurrentRequestsPerModelHighResolution`
  - Set `ScaleInCoolDown` and `ScaleOutCoolDown` values to `300` seconds

In [16]:
# Create Target Tracking Scaling Policy
target_tracking_policy_response = autoscaling_client.put_scaling_policy(
    PolicyName="SageMakerICScalingPolicy",
    ServiceNamespace="sagemaker",
    ResourceId=resource_id,
    ScalableDimension="sagemaker:inference-component:DesiredCopyCount",
    PolicyType="TargetTrackingScaling",
    TargetTrackingScalingPolicyConfiguration={
        "TargetValue": 5.0,  # Scaling triggers when endpoint receives 5 ConcurrentRequestsPerModel
        "PredefinedMetricSpecification": {
            "PredefinedMetricType": "SageMakerInferenceComponentConcurrentRequestsPerCopyHighResolution"
        },
        "ScaleInCooldown": 300,  # Cooldown period after scale-in activity
        "ScaleOutCooldown": 300,  # Cooldown period after scale-out activity
    },
)

# print(target_tracking_policy_response)
print(f"Policy ARN: [i blue]{target_tracking_policy_response['PolicyARN']}")

# print Cloudwatch Alarms
alarms = target_tracking_policy_response["Alarms"]

for alarm in alarms:
    print(f"[b]Alarm Name:[/b] [b magenta]{alarm['AlarmName']}")
    # print(f"[b]Alarm ARN:[/b] [i green]{alarm['AlarmARN']}[/i green]")
    print("===" * 15)

Policy ARN: 
arn:aws:autoscaling:us-east-1:057716757052:scalingPolicy:ffd3d714-26ed-4b47-997c-f648b07f9600:resource/sagemaker/in
ference-component/llama3-1722146573-5d08-IC-llama3b:policyName/SageMakerICScalingPolicy

Alarm Name: 
TargetTracking-inference-component/llama3-1722146573-5d08-IC-llama3b-AlarmHigh-5a1667aa-0a72-4ffd-9dee-9ba29af885ef

=============================================

Alarm Name: 
TargetTracking-inference-component/llama3-1722146573-5d08-IC-llama3b-AlarmLow-23ed4150-b5aa-4940-9b66-a5812adf9673

=============================================

## Cleanup

- Deregister scalable target. This automatically deletes associated cloudwatch alarms.
- Delete model
- Delete endpoint

In [17]:
try:
    # Deregister the scalable target for AAS
    autoscaling_client.deregister_scalable_target(
        ServiceNamespace="sagemaker",
        ResourceId=resource_id,
        ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    )
    print(f"Scalable target for [b]{resource_id}[/b] deregistered. ✅")
except autoscaling_client.exceptions.ObjectNotFoundException:
    print(f"Scalable target for [b]{resource_id}[/b] not found!.")

print("---" * 10)

try:
    print(f"Deleting inference components: [b magenta]{inference_component_name_llama3b} ✅")
    # Delete inference component
    sagemaker_client.delete_inference_component(
        InferenceComponentName=inference_component_name_llama3b
    )
except Exception as e:
    print(f"{e}")


try:
    print(f"Deleting model: [b magenta]{deployment_name}-model-llama3 ✅")
    predictor.delete_model()
except Exception as e:
    print(f"{e}")


try:
    print(f"Deleting endpoint: [b magenta]{predictor.endpoint_name} ✅")
    predictor.delete_endpoint()
except Exception as e:
    print(f"{e}")

print("---" * 10)
print("Done")

Scalable target for inference-component/llama3-1722146573-5d08-IC-llama3b not found!.

------------------------------

Deleting inference components: llama3-1722146573-5d08-IC-llama3b ✅

Deleting model: sm-model-llama3 ✅

Deleting endpoint: llama3-1722146573-5d08-endpoint ✅

------------------------------

Done

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-IC-Llama3-8B-AppAutoScaling.ipynb)